In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import os
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By

def make_dirs(path):
    try:
        if not os.path.exists(path):
            os.makedirs(path)
            print(f'{path}建立成功.')
        else:
            print(f'{path}已存在.')
    except Exception as e:
        print('建立floder失敗.', e)

def get_soup(url, post_data=None):
    try:
        if post_data is not None:
            resp = requests.post(url, post_data, headers={
                'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 11_2_3) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/12.2'})
        else:
            resp = requests.get(url, headers={
                'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 11_2_3) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/12.2'})
        resp.encoding = 'utf-8'
        if resp.status_code == 200:
            soup = BeautifulSoup(resp.text, 'lxml')
            return soup
        else:
            print('網頁取得失敗', resp.status_code)
    except Exception as e:
        print('網址錯誤', e)

def save_pic(url, filename='temp.jpg'):
    try:
        resp = requests.get(url, headers={
            'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 11_2_3) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/12.2'})
        if resp.status_code == 200:
            with open(filename, 'wb') as f:
                f.write(resp.content)
                print(f'{filename}儲存完畢')
    except Exception as e:
        print('儲存失敗', resp.status_code, e)

In [2]:
resp = get_soup("https://www.books.com.tw/web/sys_saletopb/books/?loc=P_menu_th_1_002")
soup = resp

In [3]:
books = soup.find(class_="mod_a clearfix").find_all("li", class_="item")

In [45]:
data = []

# 儲存作者及價格資訊的字典
author_prices = {}

for book in books:
    try:
        title = book.find("h4").text
        author = book.find("li").text.split("：")[1].strip()

        # 檢查 "折" 是否存在
        price_text = book.find(class_="price_a").text
        if "折" in price_text:
            price = price_text.split("折")[1].strip()
        else:
            price = price_text.split("優惠價：")[1].strip()

        link = book.find("a").get("href")
        cover = book.find(class_="cover").get("src")
        data.append([title, author, price, link, cover])

        # 更新作者及價格字典
        if author in author_prices:
            author_prices[author].append(price)
        else:
            author_prices[author] = [price]

    except Exception as e:
        print(e)

In [46]:
bb = pd.DataFrame(data, columns=["書名", "作者", "價格", "購買連結", "封面圖片連結"])

In [47]:
top_three_authors = sorted(author_prices, key=lambda x: len(author_prices[x]), reverse=True)[:3]
print(f"作品數前三名作家: {', '.join(top_three_authors)}")
bb

作品數前三名作家: 劉潤, 愛瑞克, 周慕姿


,書名,作者,價格,購買連結,封面圖片連結
0,我可能錯了：森林智者的最後一堂人生課,"比約恩．納提科．林德布勞,卡洛琳．班克勒,納維德．莫迪里",355元,https://www.books.com.tw/products/0010947051?l...,https://im2.book.com.tw/image/getImage?i=https...
1,蛤蟆先生去看心理師（暢銷300萬冊！英國心理諮商經典，附《蛤蟆先生勇氣藏書卡》組）,羅伯．狄保德,316元,https://www.books.com.tw/products/0010914794?l...,https://im1.book.com.tw/image/getImage?i=https...
2,原子習慣：細微改變帶來巨大成就的實證法則,詹姆斯‧克利爾,260元,https://www.books.com.tw/products/0010822522?l...,https://im1.book.com.tw/image/getImage?i=https...
3,窮查理的普通常識（增修版）：巴菲特50年智慧合夥人查理．蒙格的人生哲學,查理‧蒙格,355元,https://www.books.com.tw/products/0010817729?l...,https://im2.book.com.tw/image/getImage?i=https...
4,逆思維：華頓商學院最具影響力的教授，突破人生盲點的全局思考【博客來獨家書封版】,亞當．格蘭特,331元,https://www.books.com.tw/products/0010928119?l...,https://im2.book.com.tw/image/getImage?i=https...
...,...,...,...,...,...
95,親愛的，今天吃什麼？【愛妻料理的N種靈感】#食材不變×菜色百變 #保證掃盤食譜！,貝蒂做便當,379元,https://www.books.com.tw/products/0010974066?l...,https://im1.book.com.tw/image/getImage?i=https...
96,與成功有約：高效能人士的七個習慣（30週年全新增訂版）,"史蒂芬‧柯維,西恩‧柯維",434元,https://www.books.com.tw/products/0010874292?l...,https://im1.book.com.tw/image/getImage?i=https...
97,傅佩榮講易經：詳解易經六十四卦（全二冊）,傅佩榮,711元,https://www.books.com.tw/products/0010965170?l...,https://im1.book.com.tw/image/getImage?i=https...
98,達克比與世界未解之謎1：水怪貓騎士：尼斯湖水怪的疑案調查（首刷隨機贈超人氣角色書籤）,胡妙芬,284元,https://www.books.com.tw/products/0010970605?l...,https://im2.book.com.tw/image/getImage?i=https...


In [48]:
bb.to_csv("博客來中文.csv", index=False)